In [1]:
import findspark
findspark.init('/home/akshay/spark-3.5.1-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_tutorials_1").getOrCreate()

24/06/23 23:46:06 WARN Utils: Your hostname, akshay-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/23 23:46:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 23:46:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# User-Defined Functions

## Overview

User-defined functions (UDFs) are functions that are created by users to perform specific tasks or computations within a program or script. They allow for code modularity, reusability, and abstraction of complex operations into simpler, named units.

## Features

- **Modularity**: UDFs break down complex tasks into smaller, manageable parts.
- **Reusability**: Once defined, UDFs can be used multiple times within the same program or in different programs.
- **Abstraction**: UDFs hide the implementation details of a task, making the code easier to understand and maintain.

## Syntax

In most programming languages, defining a UDF follows a common syntax:

```python
# Example in Python
def function_name(parameters):
    # Function body
    # Perform operations
    return result


In [2]:
employees = [
    (1, 'John Doe', 50000, 2000),
    (2, 'Jane Smith', 60000, 2500),
    (3, 'Michael Johnson', 55000, 2200),
    (4, 'Emily Davis', 52000, 1800),
    (5, 'Chris Brown', 58000, 2700),
    (6, 'Sarah Wilson', 53000, 2000),
    (7, 'Kevin Lee', 51000, 1900),
    (8, 'Jessica Moore', 59000, 2600),
    (9, 'David Anderson', 54000, 2100),
    (10, 'Amy Taylor', 57000, 2400)
]


In [3]:
schema = ["id", "name", "salary", "bonas"]

In [4]:
df = spark.createDataFrame(data=employees, schema=schema)
df.show()

+---+---------------+------+-----+
| id|           name|salary|bonas|
+---+---------------+------+-----+
|  1|       John Doe| 50000| 2000|
|  2|     Jane Smith| 60000| 2500|
|  3|Michael Johnson| 55000| 2200|
|  4|    Emily Davis| 52000| 1800|
|  5|    Chris Brown| 58000| 2700|
|  6|   Sarah Wilson| 53000| 2000|
|  7|      Kevin Lee| 51000| 1900|
|  8|  Jessica Moore| 59000| 2600|
|  9| David Anderson| 54000| 2100|
| 10|     Amy Taylor| 57000| 2400|
+---+---------------+------+-----+



lets create custom function for totalsalary

In [5]:
def totalSalary(s,b):
    return s+b

In [6]:
from pyspark.sql.functions import udf

In [7]:
help(udf)

Help on function udf in module pyspark.sql.functions:

udf(f: Union[Callable[..., Any], ForwardRef('DataTypeOrString'), NoneType] = None, returnType: 'DataTypeOrString' = StringType(), *, useArrow: Optional[bool] = None) -> Union[ForwardRef('UserDefinedFunctionLike'), Callable[[Callable[..., Any]], ForwardRef('UserDefinedFunctionLike')]]
    Creates a user defined function (UDF).
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    f : function
        python function if used as a standalone function
    returnType : :class:`pyspark.sql.types.DataType` or str
        the return type of the user-defined function. The value can be either a
        :class:`pyspark.sql.types.DataType` object or a DDL-formatted type string.
    useArrow : bool or None
        whether to use Arrow to optimize the (de)serialization. When it is None, the
        Spark config "spark.sql.execution.pythonUDF.arrow.enabled" takes

In [10]:
from pyspark.sql.types import IntegerType

In [11]:
TotalPayment = udf(lambda s,b : totalSalary(s,b), IntegerType())

In [12]:
df1 = df.withColumn("totalPayment", TotalPayment(df.salary, df.bonas))
df1.show()

+---+---------------+------+-----+------------+
| id|           name|salary|bonas|totalPayment|
+---+---------------+------+-----+------------+
|  1|       John Doe| 50000| 2000|       52000|
|  2|     Jane Smith| 60000| 2500|       62500|
|  3|Michael Johnson| 55000| 2200|       57200|
|  4|    Emily Davis| 52000| 1800|       53800|
|  5|    Chris Brown| 58000| 2700|       60700|
|  6|   Sarah Wilson| 53000| 2000|       55000|
|  7|      Kevin Lee| 51000| 1900|       52900|
|  8|  Jessica Moore| 59000| 2600|       61600|
|  9| David Anderson| 54000| 2100|       56100|
| 10|     Amy Taylor| 57000| 2400|       59400|
+---+---------------+------+-----+------------+



you can register the function as below as well

In [13]:
@udf(returnType=IntegerType())
def totalSalary(s,b):
    return s+b

In [14]:
df.select("*", totalSalary(df.salary, df.bonas).alias("total_salary")).show()

+---+---------------+------+-----+------------+
| id|           name|salary|bonas|total_salary|
+---+---------------+------+-----+------------+
|  1|       John Doe| 50000| 2000|       52000|
|  2|     Jane Smith| 60000| 2500|       62500|
|  3|Michael Johnson| 55000| 2200|       57200|
|  4|    Emily Davis| 52000| 1800|       53800|
|  5|    Chris Brown| 58000| 2700|       60700|
|  6|   Sarah Wilson| 53000| 2000|       55000|
|  7|      Kevin Lee| 51000| 1900|       52900|
|  8|  Jessica Moore| 59000| 2600|       61600|
|  9| David Anderson| 54000| 2100|       56100|
| 10|     Amy Taylor| 57000| 2400|       59400|
+---+---------------+------+-----+------------+



In [15]:
def totalPay(s,b):
    return s+b

In [17]:
spark.udf.register("totalPaySQL",totalPay, IntegerType())

<function __main__.totalPay(s, b)>

In [18]:
df.createOrReplaceTempView("emp")

In [19]:
df2 = spark.sql("SELECT *, totalPaySQL('salary','bonas') as totalpay FROM emp")

In [20]:
df2.show()

+---+---------------+------+-----+--------+
| id|           name|salary|bonas|totalpay|
+---+---------------+------+-----+--------+
|  1|       John Doe| 50000| 2000|    NULL|
|  2|     Jane Smith| 60000| 2500|    NULL|
|  3|Michael Johnson| 55000| 2200|    NULL|
|  4|    Emily Davis| 52000| 1800|    NULL|
|  5|    Chris Brown| 58000| 2700|    NULL|
|  6|   Sarah Wilson| 53000| 2000|    NULL|
|  7|      Kevin Lee| 51000| 1900|    NULL|
|  8|  Jessica Moore| 59000| 2600|    NULL|
|  9| David Anderson| 54000| 2100|    NULL|
| 10|     Amy Taylor| 57000| 2400|    NULL|
+---+---------------+------+-----+--------+

